In [1]:
import mtgtools
import numpy as np
import pandas as pd
import pickle

from mtgtools.MtgDB import MtgDB
from requests import get
from json import loads
from shutil import copyfileobj
from tqdm import tqdm
from time import sleep

In [2]:
# load the card names to make sure no duplicates are downloaded
df = pd.read_pickle('card_df.pickle')
downloaded_card_names = set(df.CardName)

In [3]:
def download_card_images(card_list):
    # set loop
    loop = tqdm(total=len(card_list), position=0, leave=False)
    cards_not_found = set()

    # download images
    for q in card_list:
        # update the loop
        loop.update()

        # skip if it has already been downloaded
        if q in downloaded_card_names:
            continue

        try:
            # load the card from scryfall
            card = loads(get(f"https://api.scryfall.com/cards/search?q={q}").text)

            # get image url and save
            img_url = card['data'][0]['image_uris']['normal']
            f_name = "cards/" + q + ".png"
            with open(f_name, 'wb') as f:
                copyfileobj(get(img_url, stream = True).raw, f)

            
            sleep(0.11)
        except:
            cards_not_found.add(q)

    return cards_not_found

In [4]:
# create a data base of cards
db = MtgDB('my_db.fs')
sf = db.root.scryfall_cards
downloaded_cards = set()

In [5]:
# get a list of card names
sample_size = 1000
names = np.empty(sample_size, dtype=object)
prices = np.empty(sample_size, dtype=float)
samples = np.random.choice(sf, size=sample_size)

for i, card in enumerate(samples):
    names[i] = card.name
    p = card.prices['usd']
    prices[i] = float(p) if p is not None else np.nan


# download the cards
not_found = download_card_images(names)


In [6]:
new_df = pd.DataFrame()
new_df = df.from_dict({
    'CardName' : names,
    'Price' : prices
})

# save the dataframe
df = pd.concat([df, new_df])

In [7]:
df.to_pickle("card_df.pickle")